In [2]:
# SDSS_search.py :  example of generating a webpage using SDSS search results
# code snippets from Aaron Deich and Aaron Romanowsky
# -> needs pre-existing CSV-format catalog spec_sql.csv (see below),
#       which includes ra, dec, and SDSS 'objid'
dataset = int(input( 'For dr7 type (7), or for dr16 type (16):'))
run = int(input('Type which run'))
from pylab import *
from astropy.io import ascii
import pandas as pd  

# generate a local webpage for allowing visual inspection of thumbnails.
def generate_webpage_of_results(candidates_recarray, oFileNames):
    lCandidates = [] # a list of dicts, each one containing associated data for each candidate.
    for sCandidateLine in candidates_recarray:
        objid = sCandidateLine['objid']
        ra = sCandidateLine['ra']
        dec = sCandidateLine['dec']
        concentration_index = sCandidateLine['concentration_index']
        photo_z = sCandidateLine['photo_z']
        host_galaxy = sCandidateLine['host_galaxy']
        distance = sCandidateLine['distance']
        magnitude = sCandidateLine['magnitude']
        axis_ratio = sCandidateLine['axis_ratio']
        category = sCandidateLine['category']
        radius = sCandidateLine['radius']
        if dataset == 7:
            sThumbnailImg = """http://skyservice.pha.jhu.edu/DR7/ImgCutout/getjpeg.aspx?ra={}&dec={}&scale=0.40&width=120&height=120&opt=""".format(ra, dec)
            sLink = """http://skyserver.sdss.org/dr7/en/tools/chart/navi.asp?ra={}&dec={}""".format(ra, dec)
        else:
            n = 0
#         sThumbnailImg = """http://skyservice.pha.jhu.edu/DR10/ImgCutout/getjpeg.aspx?ra={}&dec={}&scale=0.40&width=120&height=120&opt=""".format(ra, dec)
#         sLink = """http://skyserver.sdss3.org/dr10/en/tools/explore/summary.aspx?id={}""".format(objid)
        if dataset == 16: 
            sThumbnailImg = """http://skyserver.sdss.org/dr16/SkyServerWS/ImgCutout/getjpeg?ra={}&dec={}&&scale=0.40&width=120&height=120&opt=""".format(ra,dec)
            sLink = """http://skyserver.sdss.org/dr16/en/tools/explore/summary.aspx?id={}""".format(objid)
        else:
            n = 0
        lCandidates.append({'thumb':sThumbnailImg, 'link':sLink, 'objid':objid, 
                        'ra':ra, 'dec': dec, 'concentration_index': concentration_index, 'photo_z': photo_z, 'host_galaxy': host_galaxy, 'distance': distance, 'magnitude': magnitude, 'axis_ratio': axis_ratio, 'category': category, 'radius':radius})
#                         'z_spec':sCandidateLine['z'],
#                         'ptype':sCandidateLine['ptype'],
#                         'sclass':sCandidateLine['sclass'], # why not just :sclass ?
#             'u_g':sCandidateLine['m_u'] - sCandidateLine['m_g'],
#             'g_r':sCandidateLine['m_g'] - sCandidateLine['m_r'],
#             'm_i':sCandidateLine['m_i']})

    with open(oFileNames['webpage'], 'w') as f:
        f.write("<head><style>table, th, td {border: 1px solid black;}</style></head><body>")
        if dataset == 7:
            f.write("SDSS DR7 possible photometric object contaminants.<br>")
        else:
            n = 0
        if dataset == 16:
            f.write("SDSS DR16 possible photometric object contaminants.<br>")
        else:
            n = 0    
        f.write("Click on thumbnails to go to their Navigate page.<br>")
        f.write("""<table style="width:50%">""")
        f.write("<tr><th><b>SDSS thumbnail</b></th><th><b>SDSS parameters</b></th><th><b>comments</b></th></tr>")
        for candidate in lCandidates:
            f.write('<tr>')
            f.write("""<td><a href='{}'><img src='{}' width=300 height=300></a></td>""".format(candidate['link'], candidate['thumb']))
#             f.write('<td>')
            f.write('<td> category: {} <br> ra, dec: {}, {} <br> objid: {} <br> photo_z: {} <br> concentration_index: {} <br> host_galaxy: {} <br> distance: {} <br> magnitude: {} <br> axis_ratio: {} <br> radius: {} <br></td>'.format(candidate['category'], candidate['ra'], candidate['dec'], candidate['objid'], candidate['photo_z'], candidate['concentration_index'], candidate['host_galaxy'], candidate['distance'], candidate['magnitude'], candidate['axis_ratio'],candidate['radius']))
#             f.write('objid, :{}<br>\n'.format(candidate['objid']))
#             f.write('i: {}<br>\n'.format(candidate['m_i']))
#             f.write('u-g, g-r: {}, {}<br>\n'.format(candidate['u_g'],candidate['g_r']))
#             f.write('redshift: {}<br>\n'.format(candidate['z_spec']))
#             f.write('ptype, sclass: {}, {}<br>\n'.format(candidate['ptype'],candidate['sclass']))
#             f.write('</td>')
            f.write('<td> <br> </td>') # blank space for writing comments
            f.write('</tr>')
        f.write("""</table></body></html>""")

    print("HTML document with thumbnails created: {}".format(oFileNames['webpage']))

'''
# Three separate searches used for QSOs, stars, galaxies, as follows:
    
# SDSS DR10 QSO search, using this form:
  http://skyserver.sdss.org/dr10/en/tools/search/sql.aspx
    
SELECT top 100
objid, ra, dec, psfmag_u-extinction_u AS m_u, psfmag_g-extinction_g AS m_g,
psfmag_r-extinction_r AS m_r, psfmag_i-extinction_i AS m_i,
psfmag_z-extinction_z AS m_z, z, 
dbo.fPhotoTypeN(type) as ptype, class as sclass
FROM SpecPhoto 
WHERE 
 (class = 'QSO') 
 
# Then the three data files are pasted together manually for reading in -> spec_sql.csv

# Now match an external catalog of CSSs (CSS_known.txt) using a file upload here,
# with a 0.033 arcmin (=2 arcsec) search radius:
  http://skyserver.sdss.org/dr10/en/tools/crossid/crossid.aspx

SELECT 
   p.objID, p.ra, p.dec,
   p.psfMag_u-extinction_u as m_u, p.psfMag_g-extinction_g as m_g,
   p.psfMag_r-extinction_r as m_r, p.psfMag_i-extinction_i as m_i,
   p.psfMag_z-extinction_z as m_z, p.petroRad_r as rad, 
   dbo.fPhotoTypeN(p.type) as ptype
FROM #upload u
      JOIN #x x ON x.up_id = u.up_id
      JOIN PhotoObj p ON p.objID = x.objID
ORDER by m_r

# -> CSS_known.csv (remove first line)

# To include photoZ's, add these lines:
   pz.z as z_phot,
   JOIN PhotozRF pz ON pz.objID = x.objID
# (this will lose a lot of objects with no PhotoZs)

# Alternative to first query above (allowing for more parameters):

SELECT top 100
p.objID, p.ra, p.dec, p.psfmag_u-p.extinction_u AS m_u, 
p.psfmag_g-p.extinction_g AS m_g,
p.psfmag_r-p.extinction_r AS m_r, p.psfmag_i-p.extinction_i AS m_i,
p.psfmag_z-p.extinction_z AS m_z, p.petroRad_r, s.z, pz.z as z_phot,
dbo.fPhotoTypeN(p.type) as ptype, s.class as sclass
FROM PhotoObj as p
    JOIN SpecObj s on s.bestObjID = p.objID
    JOIN PhotozRF pz ON pz.objID = p.objID 
    JOIN dbo.fGetNearbyObjEq(187.7,12.4,120) AS GN ON p.objID = GN.objID 
WHERE 
 (s.class = 'QSO') 
order by s.z 

# search radii of 120, 100, 22 arcmin for QSO, star, galaxy
# -> QSO_spec2.csv etc. : note that Photo-z's were calculated only for objects
#     with GALAXY photometric types, so the catalog of "stars" generated here
#     actually consists of nearby galaxies with star-like nuclear spectra

# A similar query is used above without the Photo-z components -> QSO_spec3.csv etc.,
#  using radii of 53, 53, 22.5 arcmin
'''
# read in data from CSV file:
if dataset == 7:
    if run == 1:
        
        data1 = recfromcsv('test_svm_results_v1.csv', names=True, delimiter=',')  
#         data1 = recfromcsv('voting_results_v1.csv', names=True, delimiter=',')  
#         filename = {'webpage': 'voting_results_v1.html'}
        filename = {'webpage': 'test_svm_results_v1.html'}
    if run == 2:        
        data1 = recfromcsv('voting_results_v2.csv', names=True, delimiter=',')  
        filename = {'webpage': 'voting_results_v2.html'}
    if run == 3:
        data1 = recfromcsv('voting_results_v3.csv', names=True, delimiter=',')  
        filename = {'webpage': 'voting_results_v3.html'}
if dataset == 16:
    data1 = recfromcsv('voting_results_v2.csv', names=True, delimiter=',')
    filename = {'webpage': 'results_v2_dr16.html'}
    
generate_webpage_of_results(data1, filename)
print(data1)
     


For dr7 type (7), or for dr16 type (16):16
Type which run4
HTML document with thumbnails created: results_v2_dr16.html
[(1237654879127798103, 218.09207986,  3.24653024, 1.30271, 2.01097, 3.62133, 1.61036, 0.88873, 2.35617, 2.63017, 4.90017, 8.48417, 0.41350901, 0.10687 , 0.7490013, b'IC1024', 24.2, 0.9341861, b'Possible compact galaxies',  93.45176441, -13.31027683, 21.5085 , 19.49753, 18.19482, 18.6088 , 17.88717, 15.531, 15.257, 12.987, 9.403, b'http://skyserver.sdss.org/dr16/en/tools/explore/Summary.aspx?id=1237654879127798103')
 (1237678906791755943,  21.33600278,  9.61932926, 1.25673, 2.00814, 3.51585, 1.50771, 0.85024, 2.11116, 2.26116, 4.61516, 8.50116, 0.40750604, 0.07759 , 0.8226224, b'NGC0524', 23.3, 0.9378728, b'Possible compact galaxies', 213.7767667 , -14.08114961, 20.61401, 18.60587, 17.34914, 17.75563, 17.09816, 14.987, 14.837, 12.483, 8.597, b'http://skyserver.sdss.org/dr16/en/tools/explore/Summary.aspx?id=1237678906791755943')
 (1237679479091953676,  27.27041395, 22.04